In [1]:
import numpy as np  
import cv2 
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# path to the database - change it if needed
path = 'D:\\Python\\MachineLearning\\MachineLearning\\week07\\data\\face_data\\face_data\\'
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
        'noglasses', 'normal', 'rightlight','sad',
        'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

h = im.shape[0] # hight
w = im.shape[1] # width

D = h * w
N = len(states)*15
print(N, D, h, w)

X = np.zeros((D, N))

# collect all data
count = 0
state_labels = []

# there are 15 people
for person_id in range(1, 16):
    for state in states:
        state_labels.append(state)

        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1

# Tạo mảng y chứa nhãn cho mỗi mẫu dữ liệu
y = np.array(state_labels)

165 77760 243 320


(i) Giảm số chiều dữ liệu xuống còn 100.

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Tạo một đối tượng scaler
scaler = StandardScaler()
# Initialize PCA with n_components = min(n_samples, n_features)
pca = PCA(100)

X_scale = scaler.fit_transform(X.T)
# Fit PCA to the data X
X_pca = pca.fit_transform(X_scale)  # Transpose X to fit PCA (samples as rows, features as columns)

print("Original shape:", X.shape)
print("Shape after PCA:", X_pca.shape)


Original shape: (77760, 165)
Shape after PCA: (165, 100)


Áp dụng các phương pháp phân loại nhiều lớp: Multinomial Logistic Regression, Naïve
Bayes phù hợp và ANN (đã có code) để phân loại, tỷ lệ train:test là 0.7:0.3.

In [3]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Multinomial Logistic Regression
logreg = LogisticRegression(multi_class='multinomial', max_iter=5000)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
print("Multinomial Logistic Regression Accuracy:", accuracy_logreg)

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Naive Bayes Accuracy:", accuracy_nb)


Multinomial Logistic Regression Accuracy: 0.16
Naive Bayes Accuracy: 0.16


In [4]:
def softmax(V):
    e_V = np.exp(V - np.max(V, axis = 0, keepdims = True))
    Z = e_V / e_V.sum(axis = 0)
    return Z

# cost or loss function
def cost(Y, Yhat):
    return -np.sum(Y*np.log(Yhat))/Y.shape[1]

## One-hot coding
from scipy import sparse
def convert_labels(y, C = 11):
    Y = sparse.coo_matrix((np.ones_like(y),(y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y


In [6]:
# Convert labels to integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

d0 = 100
d1 = h = 100 # size of hidden layer

d2 = C = 11
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

Y = convert_labels(y, C)
N = X.shape[0]
eta = 1 # learning rate
print(W1.shape)
print(X_pca.shape)

(100, 100)
(165, 100)


In [7]:
for i in range(1):
    ## Feedforward
    Z1 = np.dot(W1.T, X_pca.T) + b1
    print(Z1.shape)
    A1 = np.maximum(Z1, 0)
    print(A1.shape)
    Z2 = np.dot(W2.T, A1) + b2
    print(Z1.shape)
    Yhat = softmax(Z2)
    print(Yhat.shape)

(100, 165)
(100, 165)
(100, 165)
(11, 165)


In [8]:
for i in range(5000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_pca.T) + b1
    A1 = np.maximum(Z1, 0)
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = softmax(Z2)

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(Y, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - Y )/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1[Z1 <= 0] = 0 # gradient of ReLU
    dW1 = np.dot(X_pca.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2


iter 0, loss: 2.417092
iter 1000, loss: 0.333326
iter 2000, loss: 0.174947
iter 3000, loss: 0.132623
iter 4000, loss: 0.113472


In [ ]:
Z1 = np.dot(W1.T, X_pca.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2
predicted_class = np.argmax(Z2, axis=0)
acc = 100*np.mean(predicted_class == y)
print('training accuracy: %.2f %%' % (acc))

training accuracy: 9.09 %


In [ ]:
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Load and preprocess the new images
new_image_paths = ['D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.centerlight.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.glasses.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.happy.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.leftlight.png',
                   'D:\\Python\\MachineLearning\\MachineLearning\\week05\\data\\face_data\\face_data\\subject01.normal.png']
new_images = []
for path in new_image_paths:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # Resize image to match the dimensions of the original dataset
    img_resized = cv2.resize(img, (243, 320))
    new_images.append(img_resized.flatten())  # Flatten image to match the shape of data for PCA

# Convert new images to numpy array
X_new = np.array(new_images)

# Transform new images using the PCA model trained on the original dataset
X_new_pca = pca.transform(X_new)

Z1 = np.dot(W1.T, X_new_pca.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2
predicted_class = np.argmax(Z2, axis=0)
print(predicted_class)


[0 0 0 0 0]


Chay tren trainning data

In [ ]:
# Convert labels to integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y_train)

d0 = 100
d1 = h = 100 # size of hidden layer

d2 = C = 11
# initialize parameters randomly
W1 = 0.01*np.random.randn(d0, d1)
b1 = np.zeros((d1, 1))
W2 = 0.01*np.random.randn(d1, d2)
b2 = np.zeros((d2, 1))

Y = convert_labels(Y, C)
N = X_train.shape[0]
eta = 1 # learning rate

for i in range(5000):
    ## Feedforward
    Z1 = np.dot(W1.T, X_train.T) + b1
    A1 = np.maximum(Z1, 0)
    Z2 = np.dot(W2.T, A1) + b2
    Yhat = softmax(Z2)

    # print loss after each 1000 iterations
    if i %1000 == 0:
        # compute the loss: average cross-entropy loss
        loss = cost(Y, Yhat)
        print("iter %d, loss: %f" %(i, loss))

    # backpropagation
    E2 = (Yhat - Y )/N
    dW2 = np.dot(A1, E2.T)
    db2 = np.sum(E2, axis = 1, keepdims = True)
    E1 = np.dot(W2, E2)
    E1[Z1 <= 0] = 0 # gradient of ReLU
    dW1 = np.dot(X_train.T, E1.T)
    db1 = np.sum(E1, axis = 1, keepdims = True)
    # Gradient Descent update
    W1 += -eta*dW1
    b1 += -eta*db1
    W2 += -eta*dW2
    b2 += -eta*db2

Z1 = np.dot(W1.T, X_test.T) + b1
A1 = np.maximum(Z1, 0)
Z2 = np.dot(W2.T, A1) + b2
predicted_class = np.argmax(Z2, axis=0)
acc = 100*np.mean(predicted_class == y_test)
print('training accuracy: %.2f %%' % (acc))

iter 0, loss: 2.406574


C:\Users\FPT SHOP\AppData\Local\Temp\ipykernel_11748\3476936258.py:2: RuntimeWarning: invalid value encountered in subtract
  e_V = np.exp(V - np.max(V, axis = 0, keepdims = True))


iter 1000, loss: nan
iter 2000, loss: nan
iter 3000, loss: nan
iter 4000, loss: nan
training accuracy: 0.00 %
